In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from matplotlib.pyplot import imshow
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import random
import torchvision.datasets as dsets
import torchvision.transforms as transforms #should be used to convert images to pytorch tensors

#Dataset Preprocessing

In [12]:
trans = transforms.Compose([
    transforms.Resize((128, 128))
])

In [15]:
train = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Quiz 2/train', transform= trans)

In [16]:
for idx, value in enumerate(train):
  data, label, = value
  print(idx, data, label)

  #if cat
  if (label == 0):
    data.save('/content/drive/MyDrive/Colab Notebooks/Quiz 2/training_imgs/cars/%d_%d.jpeg'%(idx, label))

  elif (label == 1):
    data.save('/content/drive/MyDrive/Colab Notebooks/Quiz 2/training_imgs/planes/%d_%d.jpeg'%(idx, label))

  else:
    data.save('/content/drive/MyDrive/Colab Notebooks/Quiz 2/training_imgs/ships/%d_%d.jpeg'%(idx, label))

0 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51DFA230> 0
1 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBB4C71F280> 0
2 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51DFA2F0> 0
3 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51D91870> 0
4 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51DF9810> 0
5 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBB4D56EAD0> 0
6 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51DFB3A0> 0
7 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51D91870> 0
8 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51DFA230> 0
9 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51DFB3A0> 0
10 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51DF9810> 0
11 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51DFB010> 0
12 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51DFB3A0> 0
13 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51DF97B0> 0
14 <PIL.Image.Image image mode=RGB size=128x

In [20]:
test = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Quiz 2/test', transform=trans)

In [21]:
for idx, value in enumerate(test):
  data, label, = value
  print(idx, data, label)

  #if cat
  if (label == 0):
    data.save('/content/drive/MyDrive/Colab Notebooks/Quiz 2/testing_imgs/cars/%d_%d.jpeg'%(idx, label))

  elif (label == 1):
    data.save('/content/drive/MyDrive/Colab Notebooks/Quiz 2/testing_imgs/planes/%d_%d.jpeg'%(idx, label))

  else:
    data.save('/content/drive/MyDrive/Colab Notebooks/Quiz 2/testing_imgs/ships/%d_%d.jpeg'%(idx, label))

0 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51D915A0> 0
1 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51DF9060> 0
2 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBB4D56F280> 0
3 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51D915A0> 0
4 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51D92F20> 0
5 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBB4D56F280> 0
6 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51D91E10> 0
7 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51D92F20> 0
8 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBB4D56F280> 0
9 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51D91E10> 0
10 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51D92F20> 0
11 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBB4D56F280> 0
12 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51D91E10> 0
13 <PIL.Image.Image image mode=RGB size=128x128 at 0x7BBA51D92F20> 0
14 <PIL.Image.Image image mode=RGB size=128x

#CNN Model

In [24]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [25]:
trans = transforms.Compose([
    transforms.ToTensor()
])

In [26]:
train = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Quiz 2/training_imgs', transform=trans)

In [82]:
train_loader = torch.utils.data.DataLoader(dataset= train,
                                          batch_size= 100,
                                          shuffle = True,
                                          drop_last = True)

In [83]:
#Create CNN Model
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(3, 16, kernel_size=2, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=1, stride=(3, 1)),
    )

    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(16, 24, kernel_size=(3, 2), stride=2, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=(2, 3), stride=(2, 1)),
    )

    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(24, 48, kernel_size=2, stride=1, padding=0),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=1, stride=1),
    )

    self.fcn1 = torch.nn.Linear(48*10*62, 128, bias=True)
    torch.nn.init.xavier_uniform_(self.fcn1.weight)

    self.layer4 = torch.nn.Sequential(
        torch.nn.ReLU(),
        nn.Dropout(0.5),
    )

    self.fcn2 = torch.nn.Linear(128, 3, bias=True)
    torch.nn.init.xavier_uniform_(self.fcn2.weight)

    self.layer5 = nn.Softmax()

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)

    out = out.view(out.size(0), -1) #reduce from 2d data to 1d data
    out = self.fcn1(out)

    out = self.layer4(out)
    out = self.fcn2(out)

    out = self.layer5(out)

    return out

In [84]:
model = CNN().to(device)
test_input = (torch.Tensor(10, 3, 128, 128)).to(device) #batch_size, input channel, image size
test_out = model(test_input)

In [85]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss = nn.CrossEntropyLoss().to(device)

In [86]:
total_batch = len(train_loader)
print(total_batch)

2


In [91]:
epochs = 50
for epoch in range(epochs):
  avg_cost = 0
  for idx, data in enumerate(train_loader):
    x_train, y_train = data
    x_train = x_train.to(device)
    y_train = y_train.to(device)

    optimizer.zero_grad()
    pred = model(x_train)
    cost = loss(pred, y_train)

    correct_pred = torch.argmax(pred, 1) == y_train
    accuracy = correct_pred.float().mean()

    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print("[Epoch: {}] Cost = {} Training ACC: {:.2f}".format(epoch+1, avg_cost, accuracy*100))


[Epoch: 1] Cost = 0.7493475675582886 Training ACC: 83.00
[Epoch: 2] Cost = 0.7581145763397217 Training ACC: 79.00
[Epoch: 3] Cost = 0.7277160882949829 Training ACC: 83.00
[Epoch: 4] Cost = 0.7340302467346191 Training ACC: 84.00
[Epoch: 5] Cost = 0.6977205276489258 Training ACC: 85.00
[Epoch: 6] Cost = 0.706301212310791 Training ACC: 82.00
[Epoch: 7] Cost = 0.6999983787536621 Training ACC: 83.00
[Epoch: 8] Cost = 0.7294573187828064 Training ACC: 78.00
[Epoch: 9] Cost = 0.7186148762702942 Training ACC: 82.00
[Epoch: 10] Cost = 0.6822102069854736 Training ACC: 86.00
[Epoch: 11] Cost = 0.7159525156021118 Training ACC: 81.00
[Epoch: 12] Cost = 0.7020860910415649 Training ACC: 89.00
[Epoch: 13] Cost = 0.6823912858963013 Training ACC: 87.00
[Epoch: 14] Cost = 0.6729307174682617 Training ACC: 93.00
[Epoch: 15] Cost = 0.6666057109832764 Training ACC: 89.00
[Epoch: 16] Cost = 0.6830372214317322 Training ACC: 84.00
[Epoch: 17] Cost = 0.6738134622573853 Training ACC: 86.00
[Epoch: 18] Cost = 0.663

In [92]:
test = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Quiz 2/testing_imgs', transform=trans)

In [93]:
test_loader = torch.utils.data.DataLoader(dataset= test,
                                          batch_size= len(test), #must always be full batch size
                                          shuffle = True,
                                          drop_last = True)

In [94]:
model.eval()

with torch.no_grad():
  for idx, data in enumerate(test_loader):
    x_test, y_test = data
    x_test = x_test.to(device)
    y_test = y_test.to(device)

  pred = model(x_test)
  correct_prediction = torch.torch.argmax(pred, axis=1) == y_test
  accuracy = correct_prediction.float().mean()
  print("Testing Accuracy {:.2f}".format(accuracy.item()*100))

Testing Accuracy 75.00
